In [74]:
import requests
import urllib
from bs4 import BeautifulSoup

In [75]:
blocklist = [
    "Main_Page",
    "Help:",
    "Special:",
    "Portal:",
    "Talk:",
    "Wikipedia:",
    "Template:"
]

In [76]:
urllib.parse.unquote("/wiki/Hamilton%E2%80%93Jacobi%E2%80%93Bellman_equation")

'/wiki/Hamilton–Jacobi–Bellman_equation'

In [84]:
def parse_wiki_link(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    wiki_content_links = set()
    for link in soup.find_all('a', href=True):
        clean_link = link["href"]
        clean_link = urllib.parse.unquote(clean_link)
        if clean_link.startswith("/wiki/"):
            wiki_link = clean_link[6:]
            if any(x in wiki_link for x in blocklist):
                continue
            wiki_content_links.add(clean_link)
    return wiki_content_links

In [85]:
link = "https://en.wikipedia.org/wiki/Hamilton–Jacobi–Bellman_equation"
l = parse_wiki_link(link)

In [79]:
class UserHistory:
    def __init__(self, user_visits):
        # user_vists is a list of links in chronological order ascending
        # user_vists[-1] is the current page
        self.outgoing_links = set()
        #self.ingoing_links = set()
        for link in user_visits:
            self.outgoing_links.update(parse_wiki_link(link))
            #self.ingoing_links.update(parse_wiki_ingoing(link))
        already_visited_pages = {link.split("wikipedia.org/"[1]) for link in user_visits}

        # remove self-loops
        self.outgoing_links -= already_visited_pages
        #self.ingoing_links -= already_visited_pages
        
        # first stage of ranking: use NLP to choose top-k links from self.outgoing_links
        #   Collect text from all pages on user_visits
        #   TF-IDF for similarity between (Text in links) and (Text in all pages)
        #   For top textually similar links, crawl these target pages for text & links
        #       check link similarity (can also use text from the target page now)


        # simple baseline: for scoring a link y_t+1 given history y_1, ..., y_t of links
        # where the outgoing links included in y_i denoted O_i={x \in y_i}
        # score(y_t+1 | y_0, ..., y_t) = \sum_i=1^t 1_{y_t+1 \in O_i && y_t+1 != y_j, j=1,...,t}
        # i.e. how many times a link appeared in history pages, but was not itself in the user history
        #   problem: what about links like ISBN? Links on every page which are not interesting
        
        # simple baseline+1: composite the frequency scoring with textual similarity between text in link
        # and text in history of pages

In [88]:
def recommend_similar_by_link(user_history):
    # user_history  
    #   incorporate idf
    #   incorporate ingoing recommendations
    # return dictionary with (key,val)=(link, score)
    pass

In [89]:
def recommend_similar_by_link_text(user_history):
    # user_history  
    #   incorporate idf
    #   incorporate ingoing recommendations
    # return dictionary with (key,val)=(link, score)
    pass

In [ ]:
def recommend_similar_by_coupling(user_history, base_scores):
    # user_history  
    #   re-rank previous results
    # return dictionary with (key,val)=(link, score)
    pass

In [ ]:
def visualize_history(user_history):
    pass